<h1> ConversationBufferMemory


In [2]:
from langchain.memory import ConversationBufferMemory

In [ ]:
# Initialization of Memory

memory = ConversationBufferMemory()
memory

In [ ]:
# Saving Context (After the save_context call, the memory object contains the conversation)

memory.save_context(
    inputs={
        "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)

In [ ]:
memory

In [10]:
# History (memory.load_memory_variables({}) function returns the message history stored in the ConversationBufferMemory object)

print(memory.load_memory_variables({})["history"])

<h1> Recording/Saving conversation

In [ ]:
memory.save_context(
    inputs={"human": "사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?"},
    outputs={
        "ai": "업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
    },
)
memory.save_context(
    inputs={"human": "인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?"},
    outputs={
        "ai": "본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다."
    },
)

In [ ]:
# return_messages=True parameter means the stored history will be returned as a list of message objects 
# rather than a concatenated string

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    inputs={
        "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)

memory.save_context(
    inputs={"human": "네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?"},
    outputs={
        "ai": "감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."
    },
)


<h1> Knowledge Graph

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationKGMemory

In [ ]:
memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context(
    {"input": "이쪽은 washington에 거주중인 파월씨 입니다."},
    {"output": "파월은 누구시죠?"},
)
memory.save_context(
    {"input": "파월은 연준의장입니다."},
    {"output": "만나서 반갑습니다."},

<h1> Use in chain

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

# LLM Initialization
llm = ChatOpenAI(temperature=0)

# Prompt Setup
# {history} is past conversations stored in memory
# {input} is the current human input

template = """The following is a friendly conversation between a human and an AI. 
The AI provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""

# Prompt Template Initialization
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template)

# Conversation Chain Setup
# ConversationKGMemory is a memory system that extracts and stores knowledge-grounded facts from the conversation rather just the conversation log
conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)



